# Create Patched Groundwater Data

In [93]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from tqdm import tqdm

# Plot the patches in 3D using plotly for interactivity
import plotly.express as px

## Setup Data Directories

In [81]:
# Define data directories
# base_data_dir = '/srv/scratch/z5370003/projects/data/groundwater/FEFLOW/coastal/variable_density/'
base_data_dir = '/Users/arpitkapoor/Library/CloudStorage/OneDrive-UNSW/Shared/Projects/01_PhD/05_groundwater/data/FEFLOW/variable_density'  # Uncomment for local testing
raw_data_dir = os.path.join(base_data_dir, 'all')
filtered_data_dir = os.path.join(base_data_dir, 'filter_all_ts')

print(f"Base data directory: {base_data_dir}")
print(f"Filtered data directory: {filtered_data_dir}")
print(f"Directory exists: {os.path.exists(filtered_data_dir)}")

Base data directory: /Users/arpitkapoor/Library/CloudStorage/OneDrive-UNSW/Shared/Projects/01_PhD/05_groundwater/data/FEFLOW/variable_density
Filtered data directory: /Users/arpitkapoor/Library/CloudStorage/OneDrive-UNSW/Shared/Projects/01_PhD/05_groundwater/data/FEFLOW/variable_density/filter_all_ts
Directory exists: True


## Get and Sort Time Series Files

In [82]:
# Get and sort time series files
ts_files = sorted(os.listdir(raw_data_dir))
print(f"Total number of files: {len(ts_files)}")
print(f"First 3 files: {ts_files[:3]}")
print(f"Last 3 files: {ts_files[-3:]}")

Total number of files: 1909
First 3 files: ['0000.csv', '0001.csv', '0002.csv']
Last 3 files: ['1906.csv', '1907.csv', '1908.csv']


## Create Patches for Grid and Function Values

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial import cKDTree
from collections import defaultdict

def create_patches(x, y, z, slice_id, n_patches, slice_split=3, n_ghost_points=5):
    """
    Assigns a patch id to each point such that each patch has (almost) the same number of points.
    Also identifies neighbouring patches and finds the closest ghost points from neighbouring patches to each patch.

    Args:
        x (array-like): X coordinates of points.
        y (array-like): Y coordinates of points.
        z (array-like): Z coordinates of points.
        slice_id (array-like): Slice/group id for each point.
        n_patches (int): Number of patches (clusters) to create.
        slice_split (int): Number of slice groups.
        n_ghost_points (int): Number of ghost points to find from each neighbour.

    Returns:
        patch_ids (np.ndarray): Array of patch ids assigned to each point.
        slice_groups (np.ndarray): Array of slice group ids for each point.
        patch_neighbours (dict): Mapping from patch id to set of neighbouring patch ids.
        patch_ghost_points (dict): Mapping from patch id to dict of neighbour patch id -> indices of closest ghost points.
    """

    # Split the slices into slice_split groups
    slices = np.sort(np.unique(slice_id))
    slices_split = np.array_split(slices, slice_split)

    # Calculate the number of patches per slice
    n_patches_per_slice = n_patches // slice_split

    # Initialize the patch ids
    patch_ids = np.empty(x.shape[0], dtype=int)
    slice_groups = np.empty(x.shape[0], dtype=int)

    patch_id_offset = 1  # Start patch ids from 1

    for i, slice_group in enumerate(slices_split):
        print(f"Slice group {i+1}: {slice_group}")

        slice_idx = np.where(np.isin(slice_id, slice_group))[0]
        slice_groups[slice_idx] = i+1

        # Get the points in the slice
        x_slice = x[slice_idx]
        y_slice = y[slice_idx]
        z_slice = z[slice_idx]

        n_points = x_slice.shape[0]

        # Stack coordinates for clustering
        coords = np.stack([x_slice, y_slice, z_slice], axis=1)

        # If there are fewer points than clusters, assign each point to its own patch
        n_clusters = min(n_patches_per_slice, n_points)
        kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
        cluster_labels = kmeans.fit_predict(coords)

        # Remap cluster labels to global patch ids
        patch_ids_slice = cluster_labels + patch_id_offset
        patch_ids[slice_idx] = patch_ids_slice

        patch_id_offset += n_patches_per_slice

    # Identify neighbouring patches
    # We'll consider two patches as neighbours if any of their points are within a certain distance threshold,
    # or if their convex hulls touch, but for simplicity, let's use a k-nearest neighbour approach.

    coords_all = np.stack([x, y, z], axis=1)
    unique_patches = np.unique(patch_ids)
    patch_indices = {pid: np.where(patch_ids == pid)[0] for pid in unique_patches}

    # Build a KDTree for all points
    tree = cKDTree(coords_all)

    # Find neighbours for each patch
    patch_neighbours = defaultdict(set)
    for pid in unique_patches:
        idx = patch_indices[pid]
        # For each point in the patch, find its k nearest neighbours (excluding itself)
        dists, nbrs = tree.query(coords_all[idx], k=10)
        for row, nbr_row in zip(idx, nbrs):
            for nbr_idx in nbr_row:
                if nbr_idx == row:
                    continue
                nbr_pid = patch_ids[nbr_idx]
                if nbr_pid != pid:
                    patch_neighbours[pid].add(nbr_pid)

    # For each patch, for each neighbour, find the n_ghost_points closest points from the neighbour patch
    patch_ghost_points = defaultdict(dict)
    for pid in unique_patches:
        idx = patch_indices[pid]
        coords_patch = coords_all[idx]
        for nbr_pid in patch_neighbours[pid]:
            nbr_idx = patch_indices[nbr_pid]
            coords_nbr = coords_all[nbr_idx]
            # For each point in patch, find the closest n_ghost_points in the neighbour patch
            nbr_tree = cKDTree(coords_nbr)
            # For all points in patch, find the closest n_ghost_points in neighbour patch
            k_val = min(n_ghost_points, len(nbr_idx))
            dists, ghost_indices = nbr_tree.query(coords_patch, k=k_val)

            # 1. Flatten and extract the distances of the ghost points from patch points
            dists_flat = dists.flatten() if dists.ndim > 1 else dists
            ghost_indices_flat = ghost_indices.flatten() if ghost_indices.ndim > 1 else ghost_indices

            # 2. Sort the ghost points by indices (relative to neighbour patch)
            sorted_idx = np.argsort(ghost_indices_flat)
            ghost_indices_sorted = ghost_indices_flat[sorted_idx]
            dists_sorted = dists_flat[sorted_idx]

            # 3. Select n_ghost_points unique ghost points with shortest distances
            # For each unique ghost index, keep the first occurrence (which will be the shortest distance due to sorting)
            unique_ghost_indices, unique_pos = np.unique(ghost_indices_sorted, return_index=True)
            # Now, sort unique_pos by the corresponding distance to get the n_ghost_points with shortest distances
            if len(unique_ghost_indices) > 0:
                # Get the distances for the unique indices
                unique_dists = dists_sorted[unique_pos]
                # Sort by distance
                dist_order = np.argsort(unique_dists)
                selected_indices = unique_ghost_indices[dist_order][:n_ghost_points]
            else:
                selected_indices = np.array([], dtype=int)

            # Map to global indices
            ghost_global_indices = np.array(nbr_idx)[selected_indices]
            patch_ghost_points[pid][nbr_pid] = ghost_global_indices

    return patch_ids, slice_groups, patch_neighbours, patch_ghost_points


## Test Patching on a Single File

In [84]:
# Test interpolation on a single file first
test_file = ts_files[0]
print(f"Testing with file: {test_file}")

# Load data
res_df = pd.read_csv(os.path.join(raw_data_dir, test_file))
print(f"Data shape: {res_df.shape}")
print(f"Columns: {res_df.columns.tolist()}")
print(f"Data range - X: [{res_df.X.min():.2f}, {res_df.X.max():.2f}]")
print(f"Data range - Y: [{res_df.Y.min():.2f}, {res_df.Y.max():.2f}]")
print(f"Data range - Z: [{res_df.Z.min():.2f}, {res_df.Z.max():.2f}]")
print(f"Head range: [{res_df['head'].min():.2f}, {res_df['head'].max():.2f}]")

Testing with file: 0000.csv
Data shape: (61360, 10)
Columns: ['node', 'ts', 'time (d)', 'X', 'Y', 'Z', 'slice', 'mass_concentration', 'pressure', 'head']
Data range - X: [355702.40, 358345.95]
Data range - Y: [6456013.44, 6459170.98]
Data range - Z: [-40.00, 33.87]
Head range: [0.00, 0.98]


In [85]:
# Normalize x, y, z to a 0, 1 range
x_norm = (res_df['X'] - res_df['X'].min()) / (res_df['X'].max() - res_df['X'].min())
y_norm = (res_df['Y'] - res_df['Y'].min()) / (res_df['Y'].max() - res_df['Y'].min())
z_norm = (res_df['Z'] - res_df['Z'].min()) / (res_df['Z'].max() - res_df['Z'].min())

# Optionally, add normalized columns to the dataframe for later use
res_df['X_norm'] = x_norm
res_df['Y_norm'] = y_norm
res_df['Z_norm'] = z_norm


In [111]:
# Choose number of patches
n_patches = 100
slice_split = 5
n_ghost_points = 10

# Create patches
patch_ids, slice_groups, patch_neighbours, patch_ghost_points = create_patches(
    x=res_df.X_norm.values,
    y=res_df.Y_norm.values,
    z=res_df.Z_norm.values,
    slice_id=res_df.slice.values,
    n_patches=n_patches,
    slice_split=slice_split,
    n_ghost_points=n_ghost_points
)

# Add patch ids to dataframe
res_df['patch_id'] = patch_ids.astype(str)
res_df['slice_group'] = slice_groups.astype(str)

# Analyse ghost points
for patch_id in np.unique(patch_ids):

    print(f"\nExamining patch {patch_id}")
    print(f"Number of neighbours: {len(patch_neighbours[patch_id])}")

    # Get the neighbours and ghost points for the patch
    neighbours = patch_neighbours[patch_id]
    ghost_points_by_patch = patch_ghost_points[patch_id]

    # Plot the points in patch_id
    points_to_plot = res_df[res_df['patch_id'] == str(patch_id)]

    # Add the ghost points
    ghost_points = []
    for k, v in ghost_points_by_patch.items():
        ghost_points.extend(v)
    ghost_points = np.array(ghost_points)

    print(f"Number of ghost points: {len(ghost_points)}")
    print(f"Number of points in patch: {len(points_to_plot)}")

    # Add the ghost points to the points to plot
    points_to_plot = pd.concat([points_to_plot, res_df.loc[ghost_points]])

    # Plot the points in patch_id
    if patch_id == 1:

        # Plot the points in patch_id along with the neighbours and ghost points
        fig = px.scatter_3d(
            points_to_plot,
            x='X_norm',
            y='Y_norm',
            z='Z_norm',
            color='patch_id',
            opacity=0.8,
            size_max=10,
            title='Patches (clusters) in 3D'
        )

        # Update the traces
        fig.update_traces(marker=dict(size=3))

        # Add the title and axis labels
        fig.update_layout(
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z'
            ),
            legend_title_text='Patch ID'
        )

        fig.show()

Slice group 1: [1 2 3 4 5 6]
Slice group 2: [ 7  8  9 10 11]
Slice group 3: [12 13 14 15 16]
Slice group 4: [17 18 19 20 21]
Slice group 5: [22 23 24 25 26]

Examining patch 1
Number of neighbours: 10
Number of ghost points: 100
Number of points in patch: 492



Examining patch 2
Number of neighbours: 7
Number of ghost points: 70
Number of points in patch: 689

Examining patch 3
Number of neighbours: 5
Number of ghost points: 50
Number of points in patch: 1659

Examining patch 4
Number of neighbours: 7
Number of ghost points: 70
Number of points in patch: 1047

Examining patch 5
Number of neighbours: 7
Number of ghost points: 70
Number of points in patch: 227

Examining patch 6
Number of neighbours: 6
Number of ghost points: 60
Number of points in patch: 528

Examining patch 7
Number of neighbours: 5
Number of ghost points: 50
Number of points in patch: 522

Examining patch 8
Number of neighbours: 7
Number of ghost points: 70
Number of points in patch: 643

Examining patch 9
Number of neighbours: 6
Number of ghost points: 60
Number of points in patch: 1081

Examining patch 10
Number of neighbours: 5
Number of ghost points: 50
Number of points in patch: 713

Examining patch 11
Number of neighbours: 8
Number of ghost points: 80
Number of points

In [123]:
patch_data = []

for patch_id in np.unique(patch_ids):

    # Get the neighbours and ghost points for the patch
    neighbours = patch_neighbours[patch_id]
    ghost_points_by_patch = patch_ghost_points[patch_id]

    # Plot the points in patch_id
    patch_points = res_df[res_df['patch_id'] == str(patch_id)]

    # Add the ghost points
    ghost_points = []
    for k, v in ghost_points_by_patch.items():
        ghost_points.extend(v)
    ghost_points = res_df.loc[np.array(ghost_points)]

    # Add the data to the patch_data list
    patch_data.append({
        'patch_id': int(patch_id),
        'slice_group': int(slice_groups[patch_id]),
        'neighbour_patches': np.array(list(map(int, neighbours))),
        'ghost_nodes': ghost_points['node'].unique(),
        'core_nodes': patch_points['node'].unique(),
    })

In [125]:
import os
import json

# Define json file path
patch_data_json = os.path.join(base_data_dir, 'patches.json')

# Write the patch data to a json file
with open(patch_data_json, 'w') as f:
    json.dump(patch_data, f, indent=2, default=lambda x: x.tolist() if hasattr(x, 'tolist') else list(x) if hasattr(x, '__iter__') and not isinstance(x, str) else x)


In [126]:
# Read the patch_data_json file and validate the data

with open(patch_data_json, 'r') as f:
    loaded_patch_data = json.load(f)

# Validation: check structure and types
def validate_patch_data(patch_data_list):
    required_keys = {'patch_id', 'slice_group', 'neighbour_patches', 'ghost_nodes', 'core_nodes'}
    for i, patch in enumerate(patch_data_list):
        # Check all required keys are present
        if not required_keys.issubset(patch.keys()):
            raise ValueError(f"Patch {i} missing required keys: {required_keys - set(patch.keys())}")
        # Check types
        if not isinstance(patch['patch_id'], int):
            raise TypeError(f"Patch {i} 'patch_id' is not int")
        if not isinstance(patch['slice_group'], int):
            raise TypeError(f"Patch {i} 'slice_group' is not int")
        if not (isinstance(patch['neighbour_patches'], list) and all(isinstance(x, int) for x in patch['neighbour_patches'])):
            raise TypeError(f"Patch {i} 'neighbour_patches' is not list of int")
        if not (isinstance(patch['ghost_nodes'], list) and all(isinstance(x, (int, str)) for x in patch['ghost_nodes'])):
            raise TypeError(f"Patch {i} 'ghost_nodes' is not list of int or str")
        if not (isinstance(patch['core_nodes'], list) and all(isinstance(x, (int, str)) for x in patch['core_nodes'])):
            raise TypeError(f"Patch {i} 'core_nodes' is not list of int or str")
    print(f"Validation successful: {len(patch_data_list)} patches loaded and validated.")

validate_patch_data(loaded_patch_data)


Validation successful: 100 patches loaded and validated.


In [ ]:
# Add patch ids to dataframe for convenience
res_df['patch_id'] = np.astype(patch_ids, str)
res_df['slice_group'] = slice_groups

# Use a discrete color space for patch coloring
# We'll use Plotly's qualitative color sets, repeating as needed
discrete_colors = px.colors.qualitative.Dark24_r  # or Set1, Set2, etc.
if n_patches > len(discrete_colors):
    # Repeat the palette to cover all patches
    color_discrete_sequence = (discrete_colors * ((n_patches // len(discrete_colors)) + 1))[:n_patches]
else:
    color_discrete_sequence = discrete_colors[:n_patches]


fig = px.scatter_3d(
    res_df,
    x='X_norm',
    y='Y_norm',
    z='Z_norm',
    color='patch_id',
    color_discrete_sequence=color_discrete_sequence,
    opacity=0.3,
    size_max=10,
    title='Patches (clusters) in 3D'
)
fig.update_traces(marker=dict(size=3))
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    legend_title_text='Patch ID'
)
fig.show()
